# Cloud configuration (AWS)

In [ ]:
import os
import json
import shutil
import base64

import boto3
from botocore.exceptions import ClientError
input_dir = '../data/'
output_dir = '../results/'

## Boto3 configuration
An AWS account must be created and configured before begining with these examples. This involves 
* [registering with AWS, and then](https://aws.amazon.com/)
* [creating a user with API access](https://console.aws.amazon.com/iam/home#/users)
  * `Add User` with `Programmatic access` 
  * Give `AdminPowerUser` or other appropriate permissions
* Copy the `access key ID`  and `secret access key`
* Create the file ~/.aws/config (where ~ is your HOME directory) and add youy keys ([More details here](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html)):

```
[default]
aws_access_key_id=foo
aws_secret_access_key=bar
```

Once this file is in place with valid credentials, you should be able to connect to AWS using the Boto3 AWS Python SDK.

In [ ]:
# Test by listing users in your account
iam = boto3.client('iam')

paginator = iam.get_paginator('list_users')
for response in paginator.paginate():
    print(response)

## Creating an EC2 instance
This is a remote server that can be turned on and off as needed.
These servers are based on an Amazon Machine Image (AMI) and also need a key pair for remote connection.


In [ ]:
# Enable the Boto3 EC2 resources
ec2 = boto3.resource('ec2')

In [ ]:
key_name = 'EC2_GAMOS6_1_example2'
# Create a key-pair for accessing future EC2 instances
outfile = open(os.path.join(output_dir, '{}.pem'.format(key_name)), 'w')
key_pair = ec2.create_key_pair(KeyName=key_name)
KeyPairOut= str(key_pair.key_material)
outfile.write(KeyPairOut)
outfile.close()

In [ ]:
# Create an EC2 instance
# The Ebs dictionary is optional
# The Image ID will change. Herewe're using the latest Ubuntu 18.04LTS, but 20.04LTS will be released soon
#  - For an updated list check: https://cloud-images.ubuntu.com/locator/ec2/
# The intance type can be anything listed here: https://aws.amazon.com/ec2/instance-types/
# The SecurityGroupsIds
new_instances = ec2.create_instances(
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/sda1',
            'Ebs': {
                'DeleteOnTermination': True,
                'VolumeSize': 20, # GB
                'VolumeType': 'standard',
                'Encrypted': False,
            },
        },
    ],
    ImageId='ami-0367b500fdcac0edc', # this value will change based on the latest release and region
    MinCount=1, 
    MaxCount=1,
    KeyName=key_name,
    InstanceType="t3.large",
    #SecurityGroupIds=[security_group_id], # See below: This defines a the permissions of how traffic can flow in and out
    TagSpecifications=[{'ResourceType':'instance',
                        'Tags': [{"Key": "Name",
                                  "Value": "Run GAMOS 6_1"}]}],
    DryRun=False
    
)

### EC2 Security Groups (optional)
Security groups are used to define ingress and egress permissions from EC2 instances
Ports, protocols, and IP ranges can be used to configure these parameters
For example, traffic to websites normally travels to port 80 (http) or 433 (https), while terminal connections are commonly made over port 21 (FTP) of 22 (SSH).

The IP range can be defined using [CIDR notation](https://en.wikipedia.org/wiki/Classless_Inter-Domain_Routing), where 0.0.0.0/0 is all IPv4 addresses on the internet, [129.170.0.0/16](https://ipinfo.io/AS10755/129.170.0.0/16) is the IPv4 address space controlled by Dartmouth College, and a.b.c.d/32 would be a single public IP. 

By executing the code block below, and then running the block above using the `security_group_id` variable, the EC2 instance will limit ingress and egress based on the paramters in the security group. So, for ingress requests the system will only respond on the given port from requests originating in the IP range specified. The connection will also still require the SSH key created above. 

In [ ]:
# This code block creates 2 security groups
#  1) Only allows SSH access from Dartmouth IP space,
#  2) Allows all connections from within security group 
# Both allow all egress from the EC2 instance
ec2_client = boto3.client('ec2')

response = ec2_client.describe_vpcs()
vpc_id = response.get('Vpcs', [{}])[0].get('VpcId', '')


try:
    response = ec2_client.create_security_group(GroupName='Dartmouth_ssh4',
                                         Description='SSH from Dartmouth IP space (API)',
                                         VpcId=vpc_id)
    dartmouth_security_group_id = response['GroupId']
    print('Security Group Created %s in vpc %s.' % (dartmouth_security_group_id, vpc_id))

    data = ec2_client.authorize_security_group_ingress(
        GroupId=dartmouth_security_group_id,
        IpPermissions=[
            {'IpProtocol': 'tcp',
             'FromPort': 22,
             'ToPort': 22,
             'IpRanges': [{'CidrIp': '129.170.0.0/16'}]}
        ])
    print('Ingress Successfully Set %s' % data)
except ClientError as e:
    print(e)
    
try:
    response = ec2_client.create_security_group(GroupName='DefaultAPI4',
                                         Description='Default (API)',
                                         VpcId=vpc_id)
    default_security_group_id = response['GroupId']
    print('Security Group Created %s in vpc %s.' % (default_security_group_id, vpc_id))

    data = ec2_client.authorize_security_group_ingress(
        GroupId=default_security_group_id,
        IpPermissions=[
            {
             'IpProtocol': '-1',
            'UserIdGroupPairs': [{'GroupId': default_security_group_id}]
            }
        ])
    print('Ingress Successfully Set %s' % data)
except ClientError as e:
    print(e)

## List EC2 instances

In [ ]:
all_instances = ec2_client.describe_instances()
for instances in all_instances['Reservations']:
    for instance in instances['Instances']:
        print(instance['InstanceId'], instance['InstanceType'], instance['ImageId'], instance['KeyName'],instance['PublicDnsName'], )

## Start/stop/terminate EC2 instance
The code below allows the user to start, stop, or terminate an instance based on the `InstanceId`
The code can be un-commented by selecting the desired lines and pressing both `ctl  /`

In [ ]:
# Start instance with given ID
# ec2_client.start_instances(
#     InstanceIds=[<instance_id>],
#     DryRun=False
# )

# Stop instance with given ID
# This allows the instance to shutdown and be re-used at a later time
#ec2.stop_instances(
#    InstanceIds=[<instance_id>],
#    DryRun=True
#)

# Terminate instance with given ID
# This deletes the EC2 instance so it will need to be re-created if needed in the future
# ec2.terminate_instances(
#     InstanceIds=[<instance_id>],
#     DryRun=True
# )

## Connect to EC2 on the terminal

Once an EC2 instance is created and available, this key can be used to access the instance from the terminal, where <`key_name`> is replaced with the variable defined earlier, and `<hostname>.<region>` is the `instance['PublicDnsName']` provided when the EC2 instances were listed. 

The user will be `ubuntu` for Ubuntu AMIs, or `ec2-user` ECS-optimized AMIs (used for AWS Batch containers).

```
chmod 600 ../results/<key_name>.pem
ssh -i ../results/<key_name>.pem  ubuntu@<hostname>.<region>.compute.amazonaws.com
```


## Configure Docker on EC2
On the EC2 instance enter the following commands:

### Terminal
```
mkdir Downloads
cd Downloads
wget "http://fismed.ciemat.es/GAMOS/download/GAMOS.6.1.0/download_scripts.sh"
chmod 755 download_scripts.sh
./download_scripts.sh
mkdir ~/docker_dev
cd ~/docker_dev
cp ~/Downloads/scripts/* .
sudo apt-get update
sudo apt-get install docker.io
vim Dockerfile
```

### vim Dockerfile
This is descibed in the corresponding paper. Briefly, this file describes what is needed to configure a GAMOS container
```
FROM ubuntu:18.04
  
WORKDIR /docker_gamos


ADD . /docker_gamos
ADD fetch_and_run.sh /usr/local/bin/fetch_and_run.sh

ARG DEBIAN_FRONTEND=noninteractive
RUN apt-get update && \
    apt-get install -y --no-install-recommends apt-utils
# Add sudo permissions for admin user
RUN apt-get install -y sudo
RUN adduser --disabled-password --gecos '' docker
RUN adduser docker sudo
RUN echo '%sudo ALL=(ALL) NOPASSWD:ALL' >> /etc/sudoers
USER docker
RUN ./installMissingPackages.Docker.Ubuntu.18.04.sh
RUN sudo ./installGamos.sh /docker_gamos/gamos
RUN source gamos/GAMOS.6.1.0/config/confgamos.sh

ENTRYPOINT ["/docker_gamos/fetch_and_run.sh"]
```


### vim installMissingPackages.Docker.Ubuntu.18.04.sh
Not all of these packages are needed, but a number were added to the generic Ubuntu.18.04 file to reduce errors when comiling
```
sudo apt-get -y install curl wget unzip git vim
sudo apt-get -y install g++
sudo apt-get -y install dpkg-dev
sudo apt-get -y install binutils zlibc zopfli
sudo apt-get -y install libx11-dev
sudo apt-get -y install libxpm-dev
sudo apt-get -y install libpng12-dev libfreetype6-dev
sudo apt-get -y install libxft-dev
sudo apt-get -y install libxext-dev
sudo apt-get -y install freeglut3-dev libglfw3-dev libglfw3 libglu1-mesa-dev mesa-common-dev
sudo apt-get -y install libxmu-dev
sudo apt-get -y install libxi-dev
sudo apt-get -y install libtiff-dev
sudo apt-get -y install cmake
sudo apt-get -y install libafterimage-dev
sudo apt-get -y install build-essential
sudo apt-get -y install libjpeg8-dev
sudo apt-get -y install libtiff5-dev
sudo apt-get -y install python
sudo apt-get -y install python-dev
sudo apt-get -y install python-numpy-dev
sudo apt-get -y install libtool
sudo apt-get -y install gfortran libfftw3-dev
sudo apt-get -y install libboost-tools-dev libboost-thread1.62-dev magics++
sudo apt-get -y install libgsl0-dev
sudo apt-get -y install libcxxtools-dev
sudo apt-get -y install librte-pmd-failsafe17.11
sudo curl "https://s3.amazonaws.com/aws-cli/awscli-bundle.zip" -o "awscli-bundle.zip"
sudo unzip awscli-bundle.zip
sudo ./awscli-bundle/install -i /usr/local/aws -b /usr/local/bin/aws
```

### vim fetch_and_run.sh
Taken from the [AWS Batch blog](https://aws.amazon.com/blogs/compute/creating-a-simple-fetch-and-run-aws-batch-job/)
```bash
#!/bin/bash

# Copyright 2013-2017 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the
# License. A copy of the License is located at
#
# http://aws.amazon.com/apache2.0/
#
# or in the "LICENSE.txt" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES
# OR CONDITIONS OF ANY KIND, express or implied. See the License for the specific language governing permissions and
# limitations under the License.

# This script can help you download and run a script from S3 using aws-cli.
# It can also download a zip file from S3 and run a script from inside.
# See below for usage instructions.

PATH="/bin:/usr/bin:/sbin:/usr/sbin:/usr/local/bin:/usr/local/sbin"
BASENAME="${0##*/}"

usage () {
  if [ "${#@}" -ne 0 ]; then
    echo "* ${*}"
    echo
  fi
  cat <<ENDUSAGE
Usage:

export BATCH_FILE_TYPE="script"
export BATCH_FILE_S3_URL="s3://my-bucket/my-script"
${BASENAME} script-from-s3 [ <script arguments> ]

  - or -

export BATCH_FILE_TYPE="zip"
export BATCH_FILE_S3_URL="s3://my-bucket/my-zip"
${BASENAME} script-from-zip [ <script arguments> ]
ENDUSAGE

  exit 2
}

# Standard function to print an error and exit with a failing return code
error_exit () {
  echo "${BASENAME} - ${1}" >&2
  exit 1
}

# Check what environment variables are set
if [ -z "${BATCH_FILE_TYPE}" ]; then
  usage "BATCH_FILE_TYPE not set, unable to determine type (zip/script) of URL ${BATCH_FILE_S3_URL}"
fi

if [ -z "${BATCH_FILE_S3_URL}" ]; then
  usage "BATCH_FILE_S3_URL not set. No object to download."
fi

scheme="$(echo "${BATCH_FILE_S3_URL}" | cut -d: -f1)"
if [ "${scheme}" != "s3" ]; then
  usage "BATCH_FILE_S3_URL must be for an S3 object; expecting URL starting with s3://"
fi

# Check that necessary programs are available
which aws >/dev/null 2>&1 || error_exit "Unable to find AWS CLI executable."
which unzip >/dev/null 2>&1 || error_exit "Unable to find unzip executable."

# Create a temporary directory to hold the downloaded contents, and make sure
# it's removed later, unless the user set KEEP_BATCH_FILE_CONTENTS.
cleanup () {
   if [ -z "${KEEP_BATCH_FILE_CONTENTS}" ] \
     && [ -n "${TMPDIR}" ] \
     && [ "${TMPDIR}" != "/" ]; then
      rm -r "${TMPDIR}"
   fi
}
trap 'cleanup' EXIT HUP INT QUIT TERM
# mktemp arguments are not very portable.  We make a temporary directory with
# portable arguments, then use a consistent filename within.
TMPDIR="$(mktemp -d -t tmp.XXXXXXXXX)" || error_exit "Failed to create temp directory."
TMPFILE="${TMPDIR}/batch-file-temp"
install -m 0600 /dev/null "${TMPFILE}" || error_exit "Failed to create temp file."

# Fetch and run a script
fetch_and_run_script () {
  # Create a temporary file and download the script
  aws s3 cp "${BATCH_FILE_S3_URL}" - > "${TMPFILE}" || error_exit "Failed to download S3 script."

  # Make the temporary file executable and run it with any given arguments
  local script="./${1}"; shift
  chmod u+x "${TMPFILE}" || error_exit "Failed to chmod script."
  exec ${TMPFILE} "${@}" || error_exit "Failed to execute script."
}

# Download a zip and run a specified script from inside
fetch_and_run_zip () {
  # Create a temporary file and download the zip file
  aws s3 cp "${BATCH_FILE_S3_URL}" - > "${TMPFILE}" || error_exit "Failed to download S3 zip file from ${BATCH_FILE_S3_URL}"

  # Create a temporary directory and unpack the zip file
  cd "${TMPDIR}" || error_exit "Unable to cd to temporary directory."
  unzip -q "${TMPFILE}" || error_exit "Failed to unpack zip file."

  # Use first argument as script name and pass the rest to the script
  local script="./${1}"; shift
  [ -r "${script}" ] || error_exit "Did not find specified script '${script}' in zip from ${BATCH_FILE_S3_URL}"
  chmod u+x "${script}" || error_exit "Failed to chmod script."
  exec "${script}" "${@}" || error_exit " Failed to execute script."
}

# Main - dispatch user request to appropriate function
case ${BATCH_FILE_TYPE} in
  zip)
    if [ ${#@} -eq 0 ]; then
      usage "zip format requires at least one argument - the script to run from inside"
    fi
    fetch_and_run_zip "${@}"
    ;;

  script)
    fetch_and_run_script "${@}"
    ;;

  *)
    usage "Unsupported value for BATCH_FILE_TYPE. Expected (zip/script)."
    ;;
esac
```

### vim getGamosFiles.sh
This modifies the files provided by the GAMOS install scripts to change the location GAMOS is installed from. It also adds optical examples.

After the following
```bash
echo Installing GAMOS version $GAMOS_VER...
tar zxf $GAMOSFILES/GAMOS.$GAMOS_VER.tgz
```

Add these lines:

```bash
echo Adding GAMOS version with Tissue Optics plugin
mkdir $HOME/backup
mv $GAMOSFILES/GAMOS.$GAMOS_VER/source/GamosCore $HOME/backup
wget -N https://github.com/ethanlarochelle/GamosCore/archive/6_1.zip
unzip 6_1.zip
mv GamosCore-6_1 GamosCore
mv GamosCore $GAMOSFILES/GAMOS.$GAMOS_VER/source/
rm 6_1.zip
echo Adding Tissue Optics tutorials
wget -N https://github.com/ethanlarochelle/GAMOS_examples/archive/master.zip
unzip master.zip
mv GAMOS_examples-master TissueOptics
mv TissueOptics $GAMOSFILES/GAMOS.$GAMOS_VER/tutorials/
rm master.zip
```
Which are followed by:
```bash
cd  $GAMOSFILES/GAMOS.$GAMOS_VER/data
tar zxf initialSeeds.tgz
cd -
```

### Build Docker image
Back at the terminal run the following:
```console
cd ~/docker_dev
chmod 755 installMissingPackages.Docker.Ubuntu.18.04.sh 
chmod 755 fetch_and_run.sh 
sudo systemctl start docker
sudo systemctl enable docker
sudo docker system prune -a
sudo docker build -t gamos_6_1_tissue_optics .
```

### Screen (Optional)

Before docker installation, use screen to allow the installation to contnue if the network connection is dropped
``` console
screen -S docker_install
screen -ls # returns 5-digit id of screen session XXXXX
screen -r XXXXX
sudo docker build -t gamos_6_1_tissue_optics .
[ctl]+a d # to dettach screen session. Returns: [detached from 20449.docker_install]
exit
```

[ctl]+a K # Kills screen session

### Check Docker Container (Optional)
Start and check container status after build
```console
sudo docker run -d -it --entrypoint "/bin/bash" --name=<name> gamos_6_1_tissue_optics
sudo docker exec -it <name> /bin/bash
```
On Docker command line: 
```console
source gamos/GAMOS.6.1.0/config/confgamos.sh
```

## Create Container registry (ECR)
This is where the container image will be stored within AWS. Alternatively, it could be stored on DockerHub

In [ ]:
ecr_client = boto3.client('ecr')
ecr_name = 'monte-carlo/gamos'

In [ ]:
# Create ECR repository, or if it is alread created get the details
try:
    ecr_repository = ecr_client.create_repository(
                        repositoryName=ecr_name)
except ClientError as e:
    ecr_repository = {}
    ecr_repository['repository'] = ecr_client.describe_repositories(repositoryNames=['monte-carlo/gamos'])['repositories'][0]

In [ ]:
# Get encoded Token to allow access to push containter for 12 hours
# *** Only need to run this if AWS CLI not installed on Docker build system ***
encoded_token = ecr_client.get_authorization_token()['authorizationData'][0]['authorizationToken']

In [ ]:
# *** Only need to run this if AWS CLI not installed on Docker build system ***
# Decode token
decoded_token = base64.b64decode(encoded_token).decode('UTF-8')
decoded_token_no_user = decoded_token.split('AWS:')[1]
# Copy token to clipboard (returns 0 on success)
os.system("echo '{}' | pbcopy".format(decoded_token_no_user))

In [ ]:
# Display URI to use in cell below
ecr_repository['repository']['repositoryUri']

### Pushing container to repository
Replace `<ecr_repository['repository']['repositoryUri']>` with the output of the above code cell

On build EC2 insance

 If AWS command line interface is installed on system building docker image use the following (change region as appropriate):
 ```console
 aws ecr get-login --region us-east-2
 ```
 Else, if building on a system without AWS credentials, like a new EC2 instance, use the following with the copied token above. Replace the URI with your Repository URI.
 
```console
sudo docker login -u AWS  <ecr_repository['repository']['repositoryUri']>
<paste decoded token, above>
```

Then run the following to tag and push the container image. Replace the link as appropriate for your configuration.
```console
sudo docker tag gamos_6_1_tissue_optics:latest <ecr_repository['repository']['repositoryUri']>:latest
sudo docker push <ecr_repository['repository']['repositoryUri']>:latest
```

At this point EC2 instance used to create container can be stopped

In [ ]:
# Check to see if container image is registered
container_images = ecr_client.describe_images(
    repositoryName='monte-carlo/gamos')
container_images['imageDetails']

## Copy simulation template to S3
Template simulations are created in a single directory. This directory has everything needed to run the simulation and accept argument inputs. The folder is compressed and placed in an AWS storage bucket using S3. AWS Batch will later be configured to look in this bucket for a zip file, decompress it and run a shell script with given argument inputs.

In [ ]:
s3_client=boto3.client('s3')
s3_resource = boto3.resource('s3')

### 6MV Fluence detector

In [ ]:
# Compress simulation template
os.path.join(input_dir, '../data', '6MV_Fluence')
run_zip_name_f= 'gamos-ptg4-6mv-fluence'
archive_file_f = shutil.make_archive(run_zip_name_f, 'zip', os.path.join(input_dir, '6MV_Fluence'))
shutil.move(archive_file_f, input_dir)

In [ ]:
s3_bucket_response_f = s3_client.create_bucket(
    Bucket=run_zip_name_f,
    CreateBucketConfiguration={
        'LocationConstraint': 'us-east-2'
    }
)

In [ ]:
s3_resource.meta.client.upload_file(os.path.join(input_dir, '{}.zip'.format(run_zip_name_f)), run_zip_name_f, '{}.zip'.format(run_zip_name_f))

### 6MV No fluence detector

In [ ]:
# Compress simulation template
os.path.join(input_dir, '../data', '6MV_NoFluence')
run_zip_name_nf = 'gamos-ptg4-6mv-nofluence'
archive_file = shutil.make_archive(run_zip_name_nf, 'zip', os.path.join(input_dir, '6MV_NoFluence'))
shutil.move(archive_file, input_dir)

In [ ]:
s3_bucket_response = s3_client.create_bucket(
    Bucket=run_zip_name_nf,
    CreateBucketConfiguration={
        'LocationConstraint': 'us-east-2'
    }
)

In [ ]:
s3_resource.meta.client.upload_file(os.path.join(input_dir, '{}.zip'.format(run_zip_name_nf)), run_zip_name_nf, '{}.zip'.format(run_zip_name_nf))

## Configure Batch
AWS Batch is used to automatically organize containers on EC2 instances and execute multiple tasks in parallel

### Batch permissions
Identity access management (IAM) permissions need to be set to allow certain tasks to be automated

In [ ]:
batch_job_role_policy = {
    'Version': '2012-10-17',
    'Statement': [{'Sid': '',
    'Effect': 'Allow',
    'Principal': {'Service': 'ecs-tasks.amazonaws.com'},
    'Action': 'sts:AssumeRole'}]
}
try:
    job_role_response = iam.create_role(
        RoleName='Batch_IAM_Role',
        AssumeRolePolicyDocument=json.dumps(batch_job_role_policy),
        Description='IAM Role for running GAMOS Batch runs',
        MaxSessionDuration=14400)
except ClientError as e:
    job_role_response = iam.get_role(RoleName='Batch_IAM_Role')
    
# EC2 access
iam.attach_role_policy(
    RoleName=job_role_response['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/AmazonEC2FullAccess')

# S3 Access
iam.attach_role_policy(
    RoleName=job_role_response['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')

# API Gateway access
iam.attach_role_policy(
    RoleName=job_role_response['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator')

# EC2 Conatiner Service
iam.attach_role_policy(
    RoleName=job_role_response['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEC2ContainerServiceforEC2Role')

In [ ]:
batch_ecs_role_policy = {
    'Version': '2012-10-17',
    'Statement': [{'Sid': '',
    'Effect': 'Allow',
    'Principal': {'Service': 'ec2.amazonaws.com'},
    'Action': 'sts:AssumeRole'}]}
try:
    instance_ecs_role = iam.create_role(
        RoleName='ecsInstanceRole',
        AssumeRolePolicyDocument=json.dumps(batch_ecs_role_policy),
        MaxSessionDuration=3600)
except ClientError as e:
     instance_ecs_role = iam.get_role(RoleName='ecsInstanceRole')
    
# EC2 Conatiner Service
iam.attach_role_policy(
    RoleName=instance_ecs_role['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEC2ContainerServiceforEC2Role')

# S3 Access
iam.attach_role_policy(
    RoleName=instance_ecs_role['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

In [ ]:
try:
    conatiner_profile = iam.create_instance_profile(
        InstanceProfileName='ecsInstanceProfile')
except iam.exceptions.EntityAlreadyExistsException:
    conatiner_profile = iam.get_instance_profile(
        InstanceProfileName='ecsInstanceProfile')
try:    
    iam.add_role_to_instance_profile(
        RoleName=instance_ecs_role['Role']['RoleName'],
        InstanceProfileName=conatiner_profile['InstanceProfile']['InstanceProfileName'])
except iam.exceptions.LimitExceededException:
    pass

In [ ]:
batch_service_policy = {
    'Version': '2012-10-17',
    'Statement': [{'Sid': '',
    'Effect': 'Allow',
    'Principal': {'Service': 'batch.amazonaws.com'},
    'Action': 'sts:AssumeRole'}]
}

try:
    batch_service_role_response = iam.create_role(
    RoleName='Batch_Service_Role',
    AssumeRolePolicyDocument=json.dumps(batch_service_policy),
    Description='IAM Role for Batch service',
    MaxSessionDuration=3600)
except ClientError as e:
    batch_service_role_response = iam.get_role(RoleName='Batch_Service_Role')

# Batch Service
iam.attach_role_policy(
    RoleName=batch_service_role_response['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSBatchServiceRole')

### Create custom EC2 launch template
Since the GAMOS Docker image is large, we need to increase the default storage space

In [ ]:
custom_user_data = b"""Content-Type: multipart/mixed; boundary="==BOUNDARY=="
MIME-Version: 1.0

--==BOUNDARY==
Content-Type: text/cloud-boothook; charset="us-ascii"

# Set Docker daemon option dm.basesize so each container gets up to 50GB
cloud-init-per once docker_options echo 'OPTIONS="${OPTIONS} --storage-opt dm.basesize=50GB"' >> /etc/sysconfig/docker

--==BOUNDARY==--
"""

In [ ]:
launch_template_response = ec2_client.create_launch_template(
    LaunchTemplateName='launch-template-batch-via-boto3-4',
    LaunchTemplateData={
        'EbsOptimized': False,
        'BlockDeviceMappings':[{'DeviceName': '/dev/xvda',
            'Ebs': {'Encrypted': False,
             'DeleteOnTermination': True,
             'VolumeSize': 16,
             'VolumeType': 'gp2'}},
           {'DeviceName': '/dev/xvdcz',
            'Ebs': {'Encrypted': False,
             'DeleteOnTermination': True,
             'VolumeSize': 320,
             'VolumeType': 'gp2'}}],
#         'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
#             'DeleteOnTermination': True}],
        'ImageId': 'ami-035a1bdaf0e4bf265', # This is an ECS optimized AMI
        'KeyName': key_name,
        'Monitoring': {'Enabled': True},
        'DisableApiTermination': False,
        'InstanceInitiatedShutdownBehavior': 'stop',
        'UserData': base64.b64encode(custom_user_data).decode('UTF-8'), # This updates the space given to Docker
        'SecurityGroupIds': [default_security_group_id, dartmouth_security_group_id ],
        'TagSpecifications': [{
            'ResourceType': 'instance',
            'Tags': [{'Key': 'Name', 'Value': 'Docker_GAMOS boto3_LT'}]
        }]
    }
)

### Configure compute environment

In [ ]:
batch_client = boto3.client('batch')

In [ ]:
# First, get all subnets based on the security groups and corresponding VPCs
all_security_groups = ec2_client.describe_security_groups(GroupIds=[default_security_group_id,
                                                                    dartmouth_security_group_id])
all_vpc_ids = []
for each_sg in all_security_groups['SecurityGroups']:
    cur_id = each_sg['VpcId']
    if cur_id not in all_vpc_ids:
        all_vpc_ids.append(cur_id)

all_subnets = ec2_client.describe_subnets(Filters=[{'Name':'vpc-id', 'Values': all_vpc_ids}])['Subnets']
all_subnet_ids = []
for each_subnet in all_subnets:
    cur_subnet_id = each_subnet['SubnetId']
    if cur_subnet_id not in all_subnet_ids:
        all_subnet_ids.append(cur_subnet_id)

In [ ]:
# Define the name of the compute cluster
compute_name = 'GAMOS_6_1_Cluster-4'

In [ ]:
# Create a compute cluster
compute_env_response = batch_client.create_compute_environment(
    computeEnvironmentName=compute_name,
    type='MANAGED',
    state='ENABLED',
    computeResources={
        'type': 'EC2',
        'minvCpus': 0,
        'maxvCpus': 128,
        'desiredvCpus': 0,
        'instanceTypes': [
            'c5.2xlarge', 'c5.4xlarge', 'c5.9xlarge',
        ],
        'subnets': all_subnet_ids,
        'securityGroupIds': [default_security_group_id, dartmouth_security_group_id],
        'ec2KeyPair': key_name,
        'instanceRole': conatiner_profile['InstanceProfile']['Arn'],
        'tags': {
            'Name': 'Batch GAMOS 6_1'
        },
        'launchTemplate': {
            'launchTemplateId': launch_template_response['LaunchTemplate']['LaunchTemplateId']
        },
    },
    serviceRole=batch_service_role_response['Role']['Arn']
)

### Configure a job queue

In [ ]:
job_queue_name = 'GAMOS_6_1_queue-4'

In [ ]:
try: 
    job_queue_response = batch_client.create_job_queue(
        jobQueueName=job_queue_name,
        state='ENABLED',
        priority=80,
        computeEnvironmentOrder=[
            {
                'order': 1,
                'computeEnvironment': compute_env_response['computeEnvironmentName']
            },
        ]
    )
except ClientError as e:
    job_queue_response = batch_client.describe_job_queues(jobQueues=[job_queue_name])

In [ ]:
job_queue_response

### Create and submit jobs
The loop below will create 10 simulations, each with a different random seed. Each simulation will run for `number_events` events. The `inclusion_depth` alters the placement of the tumor inclusion by modifying the placement in the `world.geom` file. If multple depths are desired, the `inclusion_depth` value can be changed and cell block can be re-run. Every time the cell is run 10 more jobs will be added to the job queue

#### 6MV with fluence detector

In [ ]:
random_seed = 1000
number_events = 1000000
inclusion_depth = 7

for i in range(0,10):
    random_seed_str = str(random_seed+i) 
    
    current_job_definition_name = "GAMOSexample-6MV-fluence-tumor-{}mm-{}".format('_'.join(str(inclusion_depth).split('.')), str(i))
    
    gamos_job_def = {
        "jobDefinitionName": current_job_definition_name,
        "type": "container",
        "parameters": {},
        "retryStrategy": {
            "attempts": 1
        },
        "containerProperties": {
            "image": ecr_repository['repository']['repositoryUri'],
            "vcpus": 2,
            "memory": 3000,
            "command": [
                "batch_gamos.sh",
                str(random_seed_str),
                str(inclusion_depth),
                str(number_events),
                "transport.in"
            ],
            "jobRoleArn": job_role_response['Role']['Arn'],
            "volumes": [],
            "environment": [
                {
                    "name": "BATCH_FILE_S3_URL",
                    "value": "s3://{}/{}.zip".format(run_zip_name_f, run_zip_name_f)
                },
                {
                    "name": "BATCH_FILE_TYPE",
                    "value": "zip"
                }
            ],
        },
        "timeout": {
        "attemptDurationSeconds": 30000
        }
    }
    # Bunlde job definition into specific job in queue
    response = batch_client.register_job_definition(**gamos_job_def)
    gamos_job = {
        'jobName': "JOB-{}".format(current_job_definition_name),
        'jobQueue': job_queue_response['jobQueueArn'],#job_queue_response['jobQueueArn'],
        "jobDefinition": current_job_definition_name
    }
    # Submit job to Batch queue
    response = batch_client.submit_job(**gamos_job)

#### 6MV without fluence detector

In [ ]:
random_seed = 1000
number_events = 1000000
inclusion_depth = 7

for i in range(0,10):
    random_seed_str = str(random_seed+i) 
    
    current_job_definition_name = "GAMOSexample-6MV-nofluence-tumor-{}mm-{}".format('_'.join(str(inclusion_depth).split('.')), str(i))
    
    gamos_job_def = {
        "jobDefinitionName": current_job_definition_name,
        "type": "container",
        "parameters": {},
        "retryStrategy": {
            "attempts": 1
        },
        "containerProperties": {
            "image": ecr_repository['repository']['repositoryUri'],
            "vcpus": 2,
            "memory": 3000,
            "command": [
                "batch_gamos.sh",
                str(random_seed_str),
                str(inclusion_depth),
                str(number_events),
                "transport.in"
            ],
            "jobRoleArn": job_role_response['Role']['Arn'],
            "volumes": [],
            "environment": [
                {
                    "name": "BATCH_FILE_S3_URL",
                    "value": "s3://{}/{}.zip".format(run_zip_name_nf, run_zip_name_nf)
                },
                {
                    "name": "BATCH_FILE_TYPE",
                    "value": "zip"
                }
            ],
        },
        "timeout": {
        "attemptDurationSeconds": 30000
        }
    }
    # Bunlde job definition into specific job in queue
    response = batch_client.register_job_definition(**gamos_job_def)
    gamos_job = {
        'jobName': "JOB-{}".format(current_job_definition_name),
        'jobQueue': job_queue_response['jobQueueArn'],#job_queue_response['jobQueueArn'],
        "jobDefinition": current_job_definition_name
    }
    # Submit job to Batch queue
    response = batch_client.submit_job(**gamos_job)